In [5]:
import sys
# Install packages correctly
!{sys.executable} -m pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 43.7 MB 64 kB/s  eta 0:00:01     |██████████▊                     | 14.6 MB 7.0 MB/s eta 0:00:05


In [1]:
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from time import sleep
import cv2

In [2]:
mah_model = load_model('./Emotion_detect.h5')
## File created by Rainer Lienhart to detect face:
face_classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

In [3]:
plabels = ['Happy','Neutral','Sad','Surprise']

In [11]:
## 0 for in built camera, 1 for external camera
cam = cv2.VideoCapture(0)

In [12]:
while True:
    ret, frame = cam.read()
    ## convert the frame to gray
    togray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    ## detect the face using face classifier by Rainer Lienhart
    faces = face_classifier.detectMultiScale(togray,1.4,5)
    ## coordinates x,y and width and height of the face detected
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = togray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        ## if there is atleast 1 face or sum of no. face!=0
        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)
            # prediction of the frame
            prediction = mah_model.predict(roi)[0]
            label=plabels[prediction.argmax()]
            ## position where lable should appear on screen
            label_position = (x,y)
            ## make the lable on cv screen red(255,0,0) font size 4 thickness 3
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
       
    cv2.imshow('Your are feeling',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()
